# Unit test for batched augmentations

In [ ]:
import torch
from datasets.datasets import SHHSdataset
import constants
import torch.utils.data as data
from datasets.augmentations import *
import matplotlib.pyplot as plt
import numpy as np


In [7]:
rand_start = torch.randint(0, 10, (5,1))
print(rand_start)
rand_end = torch.randint(0, 10, (5,1))
print(rand_end)
ranges = torch.cat((rand_start, rand_end), dim=1)
print(ranges)
rand_start+rand_end

tensor([[9],
        [6],
        [4],
        [9],
        [9]])
tensor([[8],
        [7],
        [9],
        [2],
        [9]])
tensor([[9, 8],
        [6, 7],
        [4, 9],
        [9, 2],
        [9, 9]])


tensor([[17],
        [13],
        [13],
        [11],
        [18]])

In [22]:
ds = SHHSdataset(
    data_path=constants.SHHS_PATH_DEKSTOP,
    first_patient=1,
    num_patients=5
)
batch_size = 5
dl = data.DataLoader(dataset=ds,
                     batch_size=batch_size,
                     shuffle=False)
inputs, labels = list(iter(dl))[0]
inputs = inputs.squeeze()
inputs.size()
inputs.dtype

torch.float32

In [30]:
inputs = torch.randint(0, 10, [batch_size, 8], dtype=torch.float32)
inputs

tensor([[9., 3., 7., 1., 3., 7., 6., 4.],
        [4., 7., 7., 9., 5., 6., 9., 2.],
        [8., 2., 5., 0., 0., 8., 5., 0.],
        [8., 4., 4., 2., 3., 9., 8., 5.],
        [5., 0., 0., 0., 6., 2., 4., 7.]])

## Amplitude scale


In [31]:
# Amplitude scale:
torch.matmul(torch.diag(torch.arange(5, dtype=torch.float32)), inputs)

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 4.,  7.,  7.,  9.,  5.,  6.,  9.,  2.],
        [16.,  4., 10.,  0.,  0., 16., 10.,  0.],
        [24., 12., 12.,  6.,  9., 27., 24., 15.],
        [20.,  0.,  0.,  0., 24.,  8., 16., 28.]])

In [8]:
am = AmplitudeScale(
    mini=0.5,
    maxi=2,
    prob=1,
    batch_size=batch_size
)
am_inputs= am(inputs, inputs)
am_inputs.size()

KeyboardInterrupt: 

# Zero mask

In [34]:
masked = torch.clone(inputs)
ranges = torch.tensor([[1, 2], [0, 4], [1, 3], [2, 3], [0,5]])
indices = torch.arange(inputs.shape[1]).unsqueeze(0)
mask = ((indices >= ranges[:, 0].unsqueeze(1)) & (indices < ranges[:, 1].unsqueeze(1)))
masked.masked_fill_(mask, 0)

tensor([[9., 0., 7., 1., 3., 7., 6., 4.],
        [0., 0., 0., 0., 5., 6., 9., 2.],
        [8., 0., 0., 0., 0., 8., 5., 0.],
        [8., 4., 0., 2., 3., 9., 8., 5.],
        [0., 0., 0., 0., 0., 2., 4., 7.]])

# Gaussian noise

In [37]:
zs = torch.randn_like(inputs)
zs

tensor([[ 0.8263, -0.6738,  1.6873, -1.1548,  2.0940,  1.1151,  0.2025, -0.6751],
        [-1.6496, -0.6543, -1.2756, -0.9205, -0.0659,  1.1946,  0.3314,  1.1123],
        [ 1.8914,  1.0199,  1.1558,  0.8622,  0.2452, -0.8128,  2.3878, -0.3254],
        [-0.3726,  0.8766, -0.6009,  2.0358, -2.3234,  0.3286,  0.6097, -1.1671],
        [-1.0899, -0.0240,  0.2075, -0.7776, -0.5020, -0.6043,  0.5133,  0.9231]])

In [38]:
stdevs = torch.arange(0, batch_size, dtype=torch.float32)
noise = torch.matmul(torch.diag(stdevs), zs)
noise

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-1.6496, -0.6543, -1.2756, -0.9205, -0.0659,  1.1946,  0.3314,  1.1123],
        [ 3.7828,  2.0399,  2.3115,  1.7243,  0.4904, -1.6256,  4.7757, -0.6507],
        [-1.1177,  2.6299, -1.8028,  6.1074, -6.9701,  0.9859,  1.8290, -3.5014],
        [-4.3598, -0.0961,  0.8302, -3.1104, -2.0081, -2.4172,  2.0532,  3.6925]])

In [39]:
inputs_noise = inputs + noise
inputs_noise

tensor([[ 9.0000,  3.0000,  7.0000,  1.0000,  3.0000,  7.0000,  6.0000,  4.0000],
        [ 2.3504,  6.3457,  5.7244,  8.0795,  4.9341,  7.1946,  9.3314,  3.1123],
        [11.7828,  4.0399,  7.3115,  1.7243,  0.4904,  6.3744,  9.7757, -0.6507],
        [ 6.8823,  6.6299,  2.1972,  8.1074, -3.9701,  9.9859,  9.8290,  1.4986],
        [ 0.6402, -0.0961,  0.8302, -3.1104,  3.9919, -0.4172,  6.0532, 10.6925]])

# Timeshift

In [61]:
shifts = (1, -1, 3, 4, 5)
prev_inputs = torch.roll(inputs, shifts=1, dims=0)  # Find the previous inputs within the batch!
prev_inputs

tensor([[5., 0., 0., 0., 6., 2., 4., 7.],
        [9., 3., 7., 1., 3., 7., 6., 4.],
        [4., 7., 7., 9., 5., 6., 9., 2.],
        [8., 2., 5., 0., 0., 8., 5., 0.],
        [8., 4., 4., 2., 3., 9., 8., 5.]])

In [63]:
test = torch.cat((prev_inputs, inputs), dim=1)
test

tensor([[5., 0., 0., 0., 6., 2., 4., 7., 9., 3., 7., 1., 3., 7., 6., 4.],
        [9., 3., 7., 1., 3., 7., 6., 4., 4., 7., 7., 9., 5., 6., 9., 2.],
        [4., 7., 7., 9., 5., 6., 9., 2., 8., 2., 5., 0., 0., 8., 5., 0.],
        [8., 2., 5., 0., 0., 8., 5., 0., 8., 4., 4., 2., 3., 9., 8., 5.],
        [8., 4., 4., 2., 3., 9., 8., 5., 5., 0., 0., 0., 6., 2., 4., 7.]])

In [64]:
test[:, 8:]

tensor([[9., 3., 7., 1., 3., 7., 6., 4.],
        [4., 7., 7., 9., 5., 6., 9., 2.],
        [8., 2., 5., 0., 0., 8., 5., 0.],
        [8., 4., 4., 2., 3., 9., 8., 5.],
        [5., 0., 0., 0., 6., 2., 4., 7.]])

In [51]:

result = torch.stack([torch.roll(inputs[i], shifts[i], dims=0) for i in range(inputs.shape[0])])
result

tensor([[4., 9., 3., 7., 1., 3., 7., 6.],
        [7., 7., 9., 5., 6., 9., 2., 4.],
        [8., 5., 0., 8., 2., 5., 0., 0.],
        [3., 9., 8., 5., 8., 4., 4., 2.],
        [0., 6., 2., 4., 7., 5., 0., 0.]])

tensor([[9., 3., 7., 1., 3., 7., 6., 4.],
        [4., 7., 7., 9., 5., 6., 9., 2.],
        [8., 2., 5., 0., 0., 8., 5., 0.],
        [8., 4., 4., 2., 3., 9., 8., 5.],
        [5., 0., 0., 0., 6., 2., 4., 7.]])

# Now using the augmentations module

In [2]:
batch_size = 5
inputs = torch.randint(0, 10, [batch_size, 8], dtype=torch.float32)
aug_module = AugmentationModule(batch_size=batch_size)
print(inputs)

tensor([[4., 6., 2., 2., 4., 5., 2., 5.],
        [4., 6., 4., 2., 3., 7., 3., 5.],
        [9., 4., 2., 1., 4., 1., 8., 1.],
        [9., 9., 6., 4., 2., 4., 0., 0.],
        [7., 5., 6., 5., 3., 7., 1., 4.]])


In [4]:
inputs.unsqueeze(dim=1).size()

torch.Size([5, 1, 8])

In [3]:
# Amplitude scale
print(aug_module.amplitude_scale(inputs, torch.arange(batch_size, dtype=torch.float32)))

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 9.,  6.,  0.,  1.,  9.,  6.,  0.,  8.],
        [ 2.,  0., 18., 14., 18., 10., 12.,  8.],
        [ 3., 15.,  0., 12.,  9.,  6., 24., 12.],
        [12.,  0.,  8., 20., 32.,  8., 36., 28.]])


In [4]:
# zero_mask
print(aug_module.zero_mask(inputs.clone(), torch.tensor([[1, 2], [0, 4], [1, 3], [2, 3], [0,5]])))

tensor([[6., 0., 1., 3., 3., 5., 9., 1.],
        [0., 0., 0., 0., 9., 6., 0., 8.],
        [1., 0., 0., 7., 9., 5., 6., 4.],
        [1., 5., 0., 4., 3., 2., 8., 4.],
        [0., 0., 0., 0., 0., 2., 9., 7.]])


In [5]:
print(aug_module.gaussian_noise(inputs, 0.1*torch.arange(batch_size,dtype=torch.float32)))

tensor([[ 6.0000e+00,  6.0000e+00,  1.0000e+00,  3.0000e+00,  3.0000e+00,
          5.0000e+00,  9.0000e+00,  1.0000e+00],
        [ 8.8523e+00,  5.9306e+00,  1.0010e-01,  7.9739e-01,  9.0506e+00,
          6.0452e+00, -1.2899e-01,  7.9154e+00],
        [ 1.0035e+00, -1.9223e-04,  8.6435e+00,  6.8222e+00,  9.0296e+00,
          4.9184e+00,  5.5425e+00,  3.3551e+00],
        [ 1.1342e+00,  4.4656e+00,  3.0142e-01,  3.7147e+00,  3.2899e+00,
          1.8747e+00,  7.9218e+00,  4.2643e+00],
        [ 2.3850e+00,  2.1047e-01,  2.2008e+00,  5.2820e+00,  8.5332e+00,
          1.9901e+00,  9.2976e+00,  6.7943e+00]])


In [6]:
print(inputs)

tensor([[6., 6., 1., 3., 3., 5., 9., 1.],
        [9., 6., 0., 1., 9., 6., 0., 8.],
        [1., 0., 9., 7., 9., 5., 6., 4.],
        [1., 5., 0., 4., 3., 2., 8., 4.],
        [3., 0., 2., 5., 8., 2., 9., 7.]])


In [7]:
print(aug_module.time_shift(inputs.clone(), shifts=(0, 1, -2, 2, -2)))

tensor([[6., 6., 1., 3., 3., 5., 9., 1.],
        [1., 9., 6., 0., 1., 9., 6., 0.],
        [9., 7., 9., 5., 6., 4., 1., 5.],
        [6., 4., 1., 5., 0., 4., 3., 2.],
        [2., 5., 8., 2., 9., 7., 6., 6.]])


In [9]:
print(aug_module.augment(inputs))

tensor([[ 2.8516e+00, -1.7544e-03,  1.9029e+00,  4.7508e+00,  7.6028e+00,
          1.9039e+00,  8.5494e+00,  6.6477e+00],
        [ 9.6100e+00,  1.2185e+00,  9.6179e+00,  6.3876e+00,  3.9815e-02,
          1.1063e+00,  9.6585e+00,  6.4354e+00],
        [ 8.4366e+00,  6.6222e+00,  8.4338e+00,  4.7032e+00,  5.5719e+00,
          3.6945e+00,  1.3161e+00,  5.4018e+00],
        [ 4.7032e+00,  5.5719e+00,  3.6945e+00,  1.3161e+00,  5.4018e+00,
         -3.7884e-01,  4.1516e+00,  2.8819e+00],
        [ 5.1910e+00,  9.6100e+00,  1.2185e+00,  1.3161e+00,  5.4018e+00,
         -3.7884e-01,  4.1516e+00,  2.8819e+00]])


In [10]:
inputs

tensor([[6., 6., 1., 3., 3., 5., 9., 1.],
        [9., 6., 0., 1., 9., 6., 0., 8.],
        [1., 0., 9., 7., 9., 5., 6., 4.],
        [1., 5., 0., 4., 3., 2., 8., 4.],
        [3., 0., 2., 5., 8., 2., 9., 7.]])

In [13]:
torch.cat((inputs, inputs))

tensor([[6., 6., 1., 3., 3., 5., 9., 1.],
        [9., 6., 0., 1., 9., 6., 0., 8.],
        [1., 0., 9., 7., 9., 5., 6., 4.],
        [1., 5., 0., 4., 3., 2., 8., 4.],
        [3., 0., 2., 5., 8., 2., 9., 7.],
        [6., 6., 1., 3., 3., 5., 9., 1.],
        [9., 6., 0., 1., 9., 6., 0., 8.],
        [1., 0., 9., 7., 9., 5., 6., 4.],
        [1., 5., 0., 4., 3., 2., 8., 4.],
        [3., 0., 2., 5., 8., 2., 9., 7.]])